In [241]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

file_path = "dataR2.csv"
df = pd.read_csv(file_path)

X = df.drop(columns=["Classification"])
y = df["Classification"]


# (80%) de treino e validaçao / (20%) de teste
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 60% de treino / 20% de validação 
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

print(f"Tamanho treino:      {len(X_train)}")
print(f"Tamanho validação:   {len(X_val)}")
print(f"Tamanho teste:       {len(X_test)}")

X_train.to_csv("X_train.csv", index=False)  



Tamanho treino:      69
Tamanho validação:   23
Tamanho teste:       24


In [242]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

#--------------------------feature selection --------------------------
#Kruskal-Wallis
kruskal_selected_features = []
p_values = {}
classes = np.unique(y_train)
use_lda = X_train.columns
for feature in X_train.columns:
    groups = []
    for have_c in classes:
        groups.append(X_train.loc[y_train == have_c, feature]) #faz uma lista dependendo do valor do classification

    stat, p = kruskal(*groups) #stat fica com o valor H e p fica com o p-value
    p_values[feature] = p
    
    # Selecionar features com diferença estatisticamente significativa  (5% significance level)
    if p < 0.05:
        kruskal_selected_features.append(feature)
        print(f"Feature: {feature}, Kruskal-Wallis H-statistic: {stat}, p-value: {p}")


print("kruskal_selected_features:", kruskal_selected_features)


# Reduzir datasets apenas às features selecionadas pelo Kruskal-Wallis
X_krus_train_sel = X_train[kruskal_selected_features]
X_krus_val_sel = X_val[kruskal_selected_features]
X_krus_test_sel = X_test[kruskal_selected_features]




# ROC-AUC
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

# --- prepara dados (y_train e X_train já definidos) ---
classes = np.unique(y_train)
pos_label = classes[-1]   # define a classe positiva como a de maior valor (ex: 1 ou 2)

fnames = np.array(X_train.columns)
roc_auc = np.zeros(fnames.shape)

# --- calcular curvas ROC e AUC para cada feature ---
i = 0
for f in fnames:
    x_feat = X_train[f].to_numpy().astype(float)
    y_true = y_train.to_numpy()

    # curva ROC + AUC
    fpr, tpr, _ = roc_curve(y_true, x_feat, pos_label=pos_label)
    a = auc(fpr, tpr)
    if a < 0.5:
        a = 1 - a  # inverter direção se necessário
    roc_auc[i] = a

    # --- plot -> utilizar este codigo para relatorio  ---
    figR = go.Figure()
    figR.add_scatter(x=fpr, y=tpr, mode='lines+markers')
    figR.update_layout(autosize=False, width=700, height=700, title=dict(text=f))
    figR.update_xaxes(title_text="1 - Specificidade (FPR)", range=[-0.01, 1.01])
    figR.update_yaxes(title_text="Sensibilidade (TPR)", range=[-0.01, 1.01])
    figR.add_annotation(x=0.5, y=0.5, text=f"AUC: {a:.3f}", showarrow=False, yshift=10)
    figR.show()
    i += 1



# --- selecionar features com AUC acima do limiar (ex: 0.60) ---
roc_auc_selected_features = fnames[roc_auc > 0.60]
print("roc_auc_selected_features:", roc_auc_selected_features)

# --- reduzir os datasets ---
X_auc_train_sel = X_train[roc_auc_selected_features]
X_auc_val_sel   = X_val[roc_auc_selected_features]
X_auc_test_sel  = X_test[roc_auc_selected_features]


# Matriz de Correlação entre todas as features (treino)
import plotly.express as px

X = X_train[kruskal_selected_features].to_numpy().T
corrMat = np.corrcoef(X)
eatures = X_train[kruskal_selected_features].columns.tolist()
fig = px.imshow(
    corrMat,
    text_auto=True,
    labels=dict(x="Features", y="Features", color="Correlação"),
    x=eatures,
    y=eatures,
    width=1000,
    height=1000,
    color_continuous_scale=px.colors.sequential.gray
)
fig.update_layout(title="Matriz de Correlação entre TODAS as Features (Treino)")
fig.show()


Feature: Glucose, Kruskal-Wallis H-statistic: 21.882684786313593, p-value: 2.8983742948778834e-06
Feature: Insulin, Kruskal-Wallis H-statistic: 6.7897432455439075, p-value: 0.009168309393118787
Feature: HOMA, Kruskal-Wallis H-statistic: 10.142517584283269, p-value: 0.0014488814663522185
Feature: Resistin, Kruskal-Wallis H-statistic: 4.873490177055544, p-value: 0.027272288979371395
kruskal_selected_features: ['Glucose', 'Insulin', 'HOMA', 'Resistin']


roc_auc_selected_features: ['Glucose' 'Insulin' 'HOMA' 'Resistin']


In [243]:
#Removemos a insulina pq tem alta correlaçao com a homa mas tem um p-value mais alto 
kruskal_selected_features.remove('Insulin')
print("kruskal_selected_features after removing Insulin:", kruskal_selected_features)


kruskal_selected_features after removing Insulin: ['Glucose', 'HOMA', 'Resistin']


In [244]:
# --------------------dimensionality reduction --------------
X = X_train.to_numpy()

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Estandardizar (importante no PCA)
X_std =(X-np.mean(X,axis=0))/np.std(X,axis=0)

# Ajustar PCA
pca = PCA()
pca.fit(X_std)

print("PCA eigenvalues/Explained variance")
print(pca.explained_variance_)
print("Sum of eigenvalues="+str(np.sum(pca.explained_variance_)))
#PCA eigenvectors/Principal components
print("PCA eigenvectors/Principal components")
W=pca.components_.T
print(W)


fig = px.scatter(x= np.arange(1, len(pca.explained_variance_) + 1), y=pca.explained_variance_,labels=dict(x="PC",y="Explained Variance"))
fig.add_hline(y=1,line_width=3, line_dash="dash", line_color="red")
fig.update_traces(marker_size=10)
fig.show()

print("Variance (%) retained accourding to Kaiser: "+str(pca.explained_variance_[0]**2/(np.sum(pca.explained_variance_**2))*100))

print("Variance (%) retained accourding to Scree: "+str(np.sum(pca.explained_variance_[0:3]**2)/(np.sum(pca.explained_variance_**2))*100))



PCA eigenvalues/Explained variance
[3.39937515 1.51370481 1.35353887 1.09857836 0.63367146 0.489201
 0.33317247 0.28139814 0.02971268]
Sum of eigenvalues=9.13235294117647
PCA eigenvectors/Principal components
[[ 0.09124365  0.06631334  0.20826455  0.86173474  0.35918137  0.01295465
   0.24776193  0.1025244  -0.01549009]
 [ 0.26976908 -0.41526268  0.49064949 -0.19698031  0.05405953 -0.1228377
  -0.06355129  0.67233231  0.03409325]
 [ 0.41817988  0.229679   -0.20288689  0.07584906  0.05539931  0.59218733
  -0.52185083  0.21065684 -0.23451536]
 [ 0.44270869  0.34601931  0.03187266 -0.07051925 -0.29348134 -0.32612375
   0.35958441  0.02042474 -0.59672833]
 [ 0.47055691  0.3696771  -0.055869    0.0103968  -0.18414351 -0.15019912
   0.04584935  0.03645875  0.76077959]
 [ 0.28745844  0.03089067  0.5866371  -0.25934015  0.33409225  0.23178401
   0.0392825  -0.58145779  0.00593974]
 [-0.22461883  0.51149621 -0.09908361 -0.35312642  0.62776815 -0.02193226
   0.20820543  0.33975521  0.00672373]
 

Variance (%) retained accourding to Kaiser: 65.22068367525927
Variance (%) retained accourding to Scree: 88.49299067239575


In [245]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import plotly.express as px
import pandas as pd


X = X_train.to_numpy()
y = y_train.to_numpy()
# --- Normalizar ---

X=(X-np.mean(X,axis=0))/np.std(X,axis=0)

lda = LinearDiscriminantAnalysis()
lda.fit(X, y)
transformed = lda.transform(X)


#Plot transformed data
fig = px.scatter(x=transformed[:,0], y=np.zeros_like(transformed[:,0]), color=y.astype(str), labels=dict(x="LDA1", y="", color="Class"))
fig.update_traces(marker_size=10)


In [246]:

G1 = X_train.loc[y_train == 1, 'Glucose'].to_numpy()
G2 = X_train.loc[y_train == 2, 'Glucose'].to_numpy()
R1 = X_train.loc[y_train == 1, 'Resistin'].to_numpy()
R2 = X_train.loc[y_train == 2, 'Resistin'].to_numpy()
H1 = X_train.loc[y_train == 1, 'HOMA'].to_numpy()
H2 = X_train.loc[y_train == 2, 'HOMA'].to_numpy()

mu1=np.array([[np.mean(G1),np.mean(R1),np.mean(H1)]]).T
mu2=np.array([[np.mean(G2),np.mean(R2),np.mean(H2)]]).T


print("g1(x)="+str(mu1.T)+"x-0.5"+str(mu1.T@mu1))
print("g2(x)="+str(mu2.T)+"x-0.5"+str(mu2.T@mu2))

print("d(x)="+str((mu1-mu2).T)+"x-0.5"+str((mu1-mu2).T@(mu1+mu2)))

#------Mahalanobis-------
C1 = np.cov(np.array([G1,R1,H1]))
C2 = np.cov(np.array([G2,R2,H2]))

C=(C1+C2)/2
Ci= np.linalg.inv(C)

print("g1(x)="+str(mu1.T@Ci)+"x-0.5"+str(mu1.T@Ci@mu1))
print("g2(x)="+str(mu2.T@Ci)+"x-0.5"+str(mu2.T@Ci@mu2))

print("d(x)="+str((mu1-mu2).T@Ci)+"x-0.5"+str((mu1-mu2).T@Ci@(mu1+mu2)))
#---------------------

X1=np.array([G1,R1,H1]).T
X2=np.array([G2,R2,H2]).T
X=np.concatenate((X1,X2),axis=0)
y=np.concatenate((y_train[y_train==1],y_train[y_train==2]))

indx_1 = np.where(y == 1)[0]
indx_2 = np.where(y == 2)[0]


print("y=",y)
yp=np.ones(np.shape(y))

g1(x)=[[86.74193548 10.44332097  1.49216974]]x-0.5[[7635.45289486]]
g2(x)=[[106.5         16.15152132   3.37264328]]x-0.5[[11614.4963635]]
d(x)=[[-19.75806452  -5.70820035  -1.88047354]]x-0.5[[-3979.04346865]]
g1(x)=[[ 0.41176634 -0.03893119 -1.81814933]]x-0.5[[32.5978508]]
g2(x)=[[ 0.45881504  0.00373755 -1.83973001]]x-0.5[[42.71941628]]
d(x)=[[-0.04704871 -0.04266874  0.02158068]]x-0.5[[-10.12156548]]
y= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [247]:
# G1 = X_test.loc[y_test == 1, 'Glucose'].to_numpy()
# G2 = X_test.loc[y_test == 2, 'Glucose'].to_numpy()
# R1 = X_test.loc[y_test == 1, 'Resistin'].to_numpy()
# R2 = X_test.loc[y_test == 2, 'Resistin'].to_numpy()
# H1 = X_test.loc[y_test == 1, 'HOMA'].to_numpy()
# H2 = X_test.loc[y_test == 2, 'HOMA'].to_numpy()

# mu1=np.array([[np.mean(G1),np.mean(R1),np.mean(H1)]]).T
# mu2=np.array([[np.mean(G2),np.mean(R2),np.mean(H2)]]).T


# print("g1(x)="+str(mu1.T)+"x-0.5"+str(mu1.T@mu1))
# print("g2(x)="+str(mu2.T)+"x-0.5"+str(mu2.T@mu2))

# print("d(x)="+str((mu1-mu2).T)+"x-0.5"+str((mu1-mu2).T@(mu1+mu2)))

# #------Mahalanobis-------
# C1 = np.cov(np.array([G1,R1,H1]))
# C2 = np.cov(np.array([G2,R2,H2]))

# C=(C1+C2)/2
# Ci= np.linalg.inv(C)

# print("g1(x)="+str(mu1.T@Ci)+"x-0.5"+str(mu1.T@Ci@mu1))
# print("g2(x)="+str(mu2.T@Ci)+"x-0.5"+str(mu2.T@Ci@mu2))

# print("d(x)="+str((mu1-mu2).T@Ci)+"x-0.5"+str((mu1-mu2).T@Ci@(mu1+mu2)))
# #---------------------

# X1=np.array([G1,R1,H1]).T
# X2=np.array([G2,R2,H2]).T
# X=np.concatenate((X1,X2),axis=0)
# y=np.concatenate((y_test[y_test==1],y_test[y_test==2]))

# indx_1 = np.where(y == 1)[0]
# indx_2 = np.where(y == 2)[0]


# print("y=",y)
# yp=np.ones(np.shape(y))

In [248]:
#------Euclidean-------
dx=((mu1-mu2).T@(X.T-0.5*(mu1+mu2))).flatten()

yp[dx<0]=2

Hits=np.shape(np.where((y==yp))[0])[0]

TP= np.shape(np.where((y[indx_1]==yp[indx_1]))[0])[0]
TN= np.shape(np.where((y[indx_2]==yp[indx_2]))[0])[0]
FP=np.shape(np.where((y[indx_2]!=yp[indx_2]))[0])[0]
FN=np.shape(np.where((y[indx_1]!=yp[indx_1]))[0])[0]


SS=TP/(TP+FN)
SP=TN/(TN+FP)
PR=TP/(TP+FP)
F1Score=2*(PR*SS)/(PR+SS)
AC=(TN+TP)/(TP+TN+FP+FN)


print("Sensitivity(%)="+str(SS*100))
print("Specificity(%)="+str(SP*100))
print("Precision(%)="+str(PR*100))
print("F1Score(%)="+str(F1Score*100))
print("Accuracy(%)="+str(AC*100))



Sensitivity(%)=90.32258064516128
Specificity(%)=63.1578947368421
Precision(%)=66.66666666666666
F1Score(%)=76.71232876712328
Accuracy(%)=75.36231884057972


In [249]:
#------Mahalanobis-------
dx=((mu1-mu2).T@Ci@(X.T-0.5*(mu1+mu2))).flatten()
yp[dx<0]=2

Hits=np.shape(np.where((y==yp))[0])[0]

TP= np.shape(np.where((y[indx_1]==yp[indx_1]))[0])[0]
TN= np.shape(np.where((y[indx_2]==yp[indx_2]))[0])[0]
FP=np.shape(np.where((y[indx_2]!=yp[indx_2]))[0])[0]
FN=np.shape(np.where((y[indx_1]!=yp[indx_1]))[0])[0]

SS=TP/(TP+FN)
SP=TN/(TN+FP)
PR=TP/(TP+FP)
F1Score=2*(PR*SS)/(PR+SS)
AC=(TN+TP)/(TP+TN+FP+FN)


print("Sensitivity(%)="+str(SS*100))
print("Specificity(%)="+str(SP*100))
print("Precision(%)="+str(PR*100))
print("F1Score(%)="+str(F1Score*100))
print("Accuracy(%)="+str(AC*100))



Sensitivity(%)=87.09677419354838
Specificity(%)=68.42105263157895
Precision(%)=69.23076923076923
F1Score(%)=77.14285714285715
Accuracy(%)=76.81159420289855


In [250]:

me1=(X[indx_1,:].T-mu1)@(X[indx_1,:].T-mu1).T
me2=(X[indx_2,:].T-mu2)@(X[indx_2,:].T-mu2).T

mew = me1 + me2
mew_inv = np.linalg.inv(mew)
w = mew_inv@(mu1 - mu2)

b=-0.5*(w.T@mu1+w.T@mu2)

fig=go.Figure()
fig.add_trace(go.Scatter(x=X[indx_1,0],y=X[indx_1,1],mode='markers',name='1'))
fig.add_trace(go.Scatter(x=X[indx_2,0],y=X[indx_2,1],mode='markers',name='2'))
fig.update_traces(marker=dict(size=10))

x1=np.arange(20,130,1)

x2=-(w[0,0]/w[1,0])*x1-b/w[1,0]
fig.add_trace(go.Scatter(x=x1,y=x2.flatten(),
                         mode='lines', line=dict(dash='dash',color="gray",width=4),name='Hyperplane'))
fig.update_layout(
    autosize=False,
    width=900,
    height=800)
fig.show()

yptr=np.ones(np.shape(y))

dx=(w.T@X.T + b).flatten()
yptr[dx<0]=2


TPTr=np.shape(np.where((y[indx_1]==yptr[indx_1]))[0])[0]
TNTr=np.shape(np.where((y[indx_2]==yptr[indx_2]))[0])[0]
FPTr=np.shape(np.where((y[indx_2]!=yptr[indx_2]))[0])[0]
FNTr=np.shape(np.where((y[indx_1]!=yptr[indx_1]))[0])[0]


SSTr=TPTr/(TPTr+FNTr)
SPTr=TNTr/(TNTr+FPTr)
PRTr=TPTr/(TPTr+FPTr)
F1ScoreTr=2*(PRTr*SSTr)/(PRTr+SSTr)
ACTr=(TNTr+TPTr)/(TPTr+TNTr+FPTr+FNTr)

print("-----------Training----------------")
print("Sensitivity(%)="+str(SSTr*100))
print("Specificity(%)="+str(SPTr*100))
print("Precision(%)="+str(PRTr*100))
print("F1Score(%)="+str(F1ScoreTr*100))
print("Accuracy(%)="+str(ACTr*100))

-----------Training----------------
Sensitivity(%)=87.09677419354838
Specificity(%)=65.78947368421053
Precision(%)=67.5
F1Score(%)=76.05633802816901
Accuracy(%)=75.36231884057972
